In [1]:
from __future__ import division
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math, sys, glob, re
import scipy.constants as phco
from aces_statistics import *

%matplotlib inline

""" geocentric gravitational constant (m^3/s^2) """
GM = 3.986004418e14
""" Earth radius (m) """
Rearth = 6378000
""" Earth rotation (rad/s) """
wE = 7.292e-5

current_path = !pwd

In [2]:
""" This part of the code runs for several minutes, producing differences between theory and experiment for 
    all provided data. TODO: implement backup of the file before running, if previous version contained 
    significant amount of data. """

directories = glob.glob(current_path[0] + '/v4.3.2_mb_53896_53907/gs999/*')

f = open('differences1.dat', 'w')
f.write("# This file contains data for carrier and code (mean and STD), 3 frequencies each. The first number is the session number.\n")

for dataset_number in range(len(directories)):
    f.write(str(dataset_number)+"\t")
    print "Working on session number ", dataset_number
    
    for carrier_code in ["ca", "co"]:
        for frequency_n in ["1", "2", "3"]:
            data1 = collect_data(dataset_number, carrier_code, frequency_n)
            gs_orbit, iss_orbit = collect_trajectories(dataset_number)

            data1["gs_positions"] = map(lambda x: position_by_time(gs_orbit, x), data1['coord.time'])
            data1["gs_velocities"] = map(lambda x: velocity_by_time(gs_orbit, x), data1['coord.time'])
            data1["iss_positions"] = map(lambda x: position_by_time(iss_orbit, x), data1['coord.time'])
            data1["iss_velocities"] = map(lambda x: velocity_by_time(iss_orbit, x), data1['coord.time'])

            data1 = relativistic_effect(data1, frequency_n)
            
            cutoff = min(len(data1['difference']), 250)
            
            mean_difference = np.mean(data1['difference'][: cutoff])
            standard_deviation = np.std(data1['difference'][: cutoff])

            f.write(str(mean_difference) + "\t" + str(standard_deviation) + "\t")
    f.write("\n")
        
f.close()

Working on session number  0
Working on session number  1
Working on session number  2
Working on session number  3
Working on session number  4
Working on session number  5
Working on session number  6
Working on session number  7
Working on session number  8
Working on session number  9
Working on session number  10
Working on session number  11
Working on session number  12
Working on session number  13
Working on session number  14
Working on session number  15
Working on session number  16
Working on session number  17
Working on session number  18
Working on session number  19
Working on session number  20
Working on session number  21
Working on session number  22
Working on session number  23
Working on session number  24
Working on session number  25
Working on session number  26
Working on session number  27
Working on session number  28
Working on session number  29
Working on session number  30
Working on session number  31
Working on session number  32
Working on session n

In [3]:
""" Here we collect the generated data and form an unbiased minimal variance estimator """

filename = 'differences1.dat'
data_differences = pd.read_csv(current_path[0] + '/' + filename, delim_whitespace = True, skiprows=1, \
                   names=['session', 'mean_ca_1', 'std_ca_1', 'mean_ca_2', 'std_ca_2', 'mean_ca_3', 'std_ca_3',\
                          'mean_co_1', 'std_co_1', 'mean_co_2', 'std_co_2', 'mean_co_3', 'std_co_3'])

for carrier_code in ["ca", "co"]:
        for frequency_n in ["1", "2", "3"]:
            mean_name = 'mean_'+ carrier_code + '_' + frequency_n
            std_name = 'std_'+ carrier_code + '_' + frequency_n
            var_estimator = 1/np.sum(1/data_differences[std_name]**2)
            estimator = var_estimator*np.sum(data_differences[mean_name]/data_differences[std_name]**2)
            print 'estimator for ', carrier_code , ' frequency ', frequency_n, ' : ' , estimator, ' +/- ', np.sqrt(var_estimator)

estimator for  ca  frequency  1  :  -1.53346964403e-16  +/-  3.738318063609927e-15
estimator for  ca  frequency  2  :  -5.84388190477e-15  +/-  3.7105361987371075e-15
estimator for  ca  frequency  3  :  -5.97065204363e-15  +/-  5.290844808229772e-15
estimator for  co  frequency  1  :  -1.62561958368e-16  +/-  3.7532529565197865e-15
estimator for  co  frequency  2  :  -5.85627863804e-15  +/-  3.752447133435046e-15
estimator for  co  frequency  3  :  -5.96038046026e-15  +/-  5.259950060786446e-15
